In [ ]:
import gymnasium as gym
from easydict import EasyDict
from models import DQNAgent, ReplayBuffer
from pytorch_lightning.utilities.seed import seed_everything

In [ ]:
seed_everything(45)

In [ ]:
train_env = gym.make("LunarLander-v2")

In [ ]:
args = {
    "state_dim": train_env.observation_space.shape[0],
    "action_dim": train_env.action_space.shape[0],
    "batch_size": 64,
    "gamma": 0.5,
    "DDQN": False
}
agent = DQNAgent(**args)
buffer = ReplayBuffer(args["state_dim"], max_size=1e6)